In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#Initial load chicago_table_id = "opendatadbt.311.chicago311" chicago311 = requests.get("https://data.cityofchicago.org/resource/v6vf-nfxy.json? limit=10000& where=created_date >= '2021-03-18T00:00:00.000'" ) chicago311json = chicago311.json() chicago_df = pd.DataFrame(chicago311json) chicago_df = chicago_df[['created_date','closed_date','owner_department','sr_type','zip_code','street_address','city','status','latitude','longitude']] chicago_df

In [3]:
#Incremental load
chicago_table_id = "opendatadbt.311.chicago311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(created_date), max(closed_date)
FROM `opendatadbt.311.chicago311`
limit 10
"""

chicagomaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chicagomaxopendatedate = str(chicagomaxopendatedataframe['f0_'][0])

chicagomaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chicagomaxclosedate = str(chicagomaxclosedatedataframe['f1_'][0])


In [4]:
chicagomaxclosedate

'2021-03-27T09:22:15.000'

In [5]:
chicago311 = requests.get("https://data.cityofchicago.org/resource/v6vf-nfxy.json?$limit=50000&$where=created_date > '" + chicagomaxopendatedate +"' OR closed_date between '" +chicagomaxclosedate+ "' AND '" + current_datetime + "'") 
chicago311json = chicago311.json()
cols = ['sr_number','created_date','closed_date','owner_department','sr_type','zip_code','street_address','city','status','latitude','longitude']
chicago_df = pd.DataFrame(chicago311json, columns = cols)

chicago_df

,sr_number,created_date,closed_date,owner_department,sr_type,zip_code,street_address,city,status,latitude,longitude
0,SR21-00461745,2021-03-20T23:07:49.000,2021-03-27T17:50:04.000,BACP - Business Affairs and Consumer Protection,Business Complaints / Reopening Issue,60613,1025 W MONTROSE AVE,NaN,Completed,41.96174251,-87.655828486
1,SR21-00464101,2021-03-21T16:01:04.000,2021-03-27T17:50:07.000,BACP - Business Affairs and Consumer Protection,Business Complaints / Reopening Issue,60610,1448 N WELLS ST,NaN,Completed,41.909100415,-87.634909853
2,SR21-00465077,2021-03-21T19:44:01.000,2021-03-27T17:55:06.000,BACP - Business Affairs and Consumer Protection,Business Complaints / Reopening Issue,60644,5464 W MADISON ST,NaN,Completed,41.880310288,-87.762722867
3,SR21-00467547,2021-03-22T10:00:46.000,2021-03-28T06:45:39.000,Streets and Sanitation,Sanitation Code Violation,60639,4812 W NORTH AVE,NaN,Completed,41.909963609,-87.746581539
4,SR21-00468641,2021-03-22T11:28:31.000,2021-03-27T14:30:04.000,BACP - Business Affairs and Consumer Protection,Tobacco - General Complaint,60636,6700 S ASHLAND AVE,NaN,Completed,41.772199852,-87.664026881
...,...,...,...,...,...,...,...,...,...,...,...
4082,SR21-00506363,2021-03-28T09:59:37.000,NaN,Streets and Sanitation,Graffiti Removal Request,60657,3330 N LAKE SHORE DR,NaN,Open,41.943201457,-87.639993248
4083,SR21-00506226,2021-03-28T09:36:47.000,2021-03-28T09:36:48.000,Aviation,Aircraft Noise Complaint,60666,10510 W ZEMKE RD,Chicago,Completed,NaN,NaN
4084,SR21-00426423,2021-03-15T09:34:51.000,2021-03-27T17:55:04.000,BACP - Business Affairs and Consumer Protection,Business Complaints / Reopening Issue,60660,1163 W GRANVILLE AVE,NaN,Completed,41.994483984,-87.659820981
4085,SR21-00420997,2021-03-13T21:42:21.000,2021-03-27T18:00:06.000,BACP - Business Affairs and Consumer Protection,Business Complaints / Reopening Issue,60638,5446 S ARCHER AVE,NaN,Completed,41.79843138,-87.744754083


In [6]:
chicago_table_id = "opendatadbt.311.chicago311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('chicago311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    chicago_df, table_ref, job_config=job_config
).result()  # Make an API request.
